In [2]:
from __future__ import annotations

from typing import TYPE_CHECKING

from goob_ai.agent import AiAgent

import pytest

from loguru import logger as LOGGER
if TYPE_CHECKING:
    from _pytest.fixtures import FixtureRequest
    from _pytest.monkeypatch import MonkeyPatch

    from pytest_mock.plugin import MockerFixture

import asyncio
import sys
import uuid
from functools import partial
from itertools import cycle
from typing import (
    Any,
    AsyncIterator,
    Dict,
    Iterable,
    Iterator,
    List,
    Optional,
    Sequence,
    cast,
)

import pytest

from langchain_core.callbacks import CallbackManagerForRetrieverRun, Callbacks
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.language_models import FakeStreamingListLLM, GenericFakeChatModel
from langchain_core.load import dumpd
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import (
    ConfigurableField,
    Runnable,
    RunnableConfig,
    RunnableGenerator,
    RunnableLambda,
    chain,
    ensure_config,
)
from langchain_core.runnables.config import get_callback_manager_for_config
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables.schema import StreamEvent
from langchain_core.runnables.utils import Input, Output
from langchain_core.tools import tool
from langchain_core.utils.aiter import aclosing

from dotenv import load_dotenv
import os
from langchain.agents import AgentExecutor

# Load environment variables from .env file
load_dotenv()
import rich
from goob_ai import debugger
from goob_ai.agent import AiAgent


class AnyStr(str):
    def __eq__(self, other: Any) -> bool:
        return isinstance(other, str)


def _with_nulled_run_id(events: Sequence[StreamEvent]) -> List[StreamEvent]:
    """Removes the run ids from events."""
    for event in events:
        assert "run_id" in event, f"Event {event} does not have a run_id."
        assert "parent_ids" in event, f"Event {event} does not have parent_ids."
        assert isinstance(
            event["run_id"], str
        ), f"Event {event} run_id is not a string."
        assert isinstance(
            event["parent_ids"], list
        ), f"Event {event} parent_ids is not a list."

    return cast(
        List[StreamEvent],
        [{**event, "run_id": "", "parent_ids": []} for event in events],
    )


async def _as_async_iterator(iterable: List) -> AsyncIterator:
    """Converts an iterable into an async iterator."""
    for item in iterable:
        yield item


async def _collect_events(
    events: AsyncIterator[StreamEvent], with_nulled_ids: bool = True
) -> List[StreamEvent]:
    """Collect the events and remove the run ids."""
    materialized_events = [event async for event in events]

    if with_nulled_ids:
        events_ = _with_nulled_run_id(materialized_events)
    else:
        events_ = materialized_events
    for event in events_:
        event["tags"] = sorted(event["tags"])
    return events_

In [3]:
agent = AiAgent()

In [4]:
# rich.inspect(agent, all=True)
rich.print(agent.agent)

RunnableSequence(
    first=RunnableParallel(
        steps__={
            'input': RunnableLambda(...),
            'agent_scratchpad': RunnableLambda(...),
            'chat_history': RunnableLambda(...)
        }
    ),
    middle=[
        ChatPromptTemplate(
            input_variables=['agent_scratchpad', 'chat_history', 'input'],
            input_types={
                'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, 
langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, 
langchain_core.messages.tool.ToolMessage]],
                'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, 
langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, 
langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, 
langchain_core.messages.tool.ToolMessage]]
            },
            messages=[
                SystemMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=[],
                        template="\n             You are a helpful AI assistant called Malcolm Jones Developer 
Assitant 'GOOBS' as an homage to his frenchbulldog Gaston aka GOOBS.\n             Use the following pieces of 
context to answer the question at the end.\n             You were created by Tony Dark himself, Malcolm Jones to 
help our developers build better software faster.\n             You think step by step about the user request and 
provide a helpful and truthful response.\n\n             Very Important: If the question is about writing code use 
backticks (```) at the front and end of the code snippet and include the language use after the first ticks.\n\n   
If the user provides an image use Custom Tool vision_api to get more information about the image then pass the text
along with the original question to vector_store_tool.\n             You remember 10 previous messages from the 
chat thread.\n             If you use documents from any tools to provide a helpful answer to user question, please
make sure to also return a valid URL of the document you used\n             If you don't know the answer, just say 
so, and don't try to make anything up. DO NOT allow made up or fake answers.\n             If the question is not 
related to the context, politely respond that you are tuned to only answer questions that are related to the 
context.\n             Use as much detail when as possible when responding.\n             All answers should be in 
MARKDOWN (.md) Format:\n             You have a geeky and clever sense of humor.\n             "
                    )
                ),
                MessagesPlaceholder(variable_name='chat_history'),
                HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
                MessagesPlaceholder(variable_name='agent_scratchpad')
            ]
        ),
        RunnableBinding(
            bound=ChatOpenAI(
                name='ChatOpenAI',
                client=<openai.resources.chat.completions.Completions object at 0x30f4c54b0>,
                async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x30f4c7250>,
                model_name='gpt-4o',
                temperature=0.0,
                openai_api_key=SecretStr('**********'),
                openai_proxy='',
                streaming=True
            ),
            kwargs={
                'tools': [
                    {
                        'type': 'function',
                        'function': {
                            'name': 'vision_api',
                            'description': "This tool calls OpenAI's Vision API to get more information about an 
image given a URL to an image file.",
                            'parameters': {
                    

In [5]:
import langchain

agent_input = {"user name": "bossjones", "message": "how do i resize a jpeg image using opencv?", "surface_info": {"surface": "discord", "type": "channel", "source": "gpt"}}

session_id = "1111_test"

agent_executor: langchain.agents.AgentExecutor = agent.setup_agent_executor(session_id, str(agent_input))
config = {"metadata": {"session_id": session_id}}
# result = agent_executor.invoke({"input": str(agent_input)}, config=config)
# LOGGER.error(f"type(result) = {type(result)}")
# res = result.get("output", "No response generated by the agent.")

2024-07-23 21:25:59.128 | DEBUG    | goob_ai.agent:setup_agent_executor:184 - session_id = 1111_test
2024-07-23 21:25:59.130 | DEBUG    | goob_ai.agent:setup_agent_executor:185 - user_task = {'user name': 'bossjones', 'message': 'how do i resize a jpeg image using opencv?', 'surface_info': {'surface': 'discord', 'type': 'channel', 'source': 'gpt'}}


In [6]:
# agent_executor.astream_events

events = []
async for event in agent_executor.astream_events({"input": str(agent_input)}, config=config, version="v2"):
    events.append(event)

/Users/malcolm/dev/bossjones/goob_ai/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(
2024-07-23 21:25:59.204 | INFO     | langchain.callbacks.tracers.logging:callback:27 - [chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]




> Entering new AgentExecutor chain...

Invoking: `chroma_question_answering` with `{'question': 'How do I resize a JPEG image using OpenCV?'}`


To resize a JPEG image using OpenCV, you can use the `cv.resize()` function. First, read the image using `cv.imread()`, then apply `cv.resize()` with the desired dimensions, and finally save the resized image using `cv.imwrite()`. Here is a concise example:

```python
import cv2 as cv
img = cv.imread('image.jpg')
resized_img = cv.resize(img, (width, height))
cv.imwrite('resized_image.jpg', resized_img)
```

2024-07-23 21:26:08.030 | INFO     | langchain.callbacks.tracers.logging:callback:27 - [chain/end] [chain:AgentExecutor] [8.83s] Exiting Chain run with output:
[outputs]


To resize a JPEG image using OpenCV, you can use the `cv.resize()` function. Here's a detailed example in Python:

```python
import cv2 as cv

# Read the image
img = cv.imread('image.jpg')

# Define the new dimensions
width = 800  # Replace with your desired width
height = 600  # Replace with your desired height
new_dimensions = (width, height)

# Resize the image
resized_img = cv.resize(img, new_dimensions)

# Save the resized image
cv.imwrite('resized_image.jpg', resized_img)
```

In this example:
1. We read the image using `cv.imread()`.
2. We define the new dimensions for the image.
3. We resize the image using `cv.resize()`.
4. We save the resized image using `cv.imwrite()`.

Make sure to replace `'image.jpg'` with the path to your JPEG image and adjust the `width` and `height` to your desired dimensions.

For more detailed information, you can refer to the [OpenCV documentation](https://docs.opencv.org/4.x/da/d6e/tutorial_py_geometric_transformations.html).

> Finished chain.


In [10]:
# rich.print(events)

events = await _collect_events(
    agent_executor.astream_events({"input": str(agent_input)}, config=config, version="v2")  # type: ignore
)

2024-07-23 21:29:18.664 | INFO     | langchain.callbacks.tracers.logging:callback:27 - [chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]




> Entering new AgentExecutor chain...

Invoking: `chroma_question_answering` with `{'question': 'How do I resize a JPEG image using OpenCV?'}`


To resize a JPEG image using OpenCV, you can use the `cv.resize()` function. First, read the image using `cv.imread()`, then apply `cv.resize()` with the desired dimensions, and finally save the resized image using `cv.imwrite()`. Here is a concise example:

```python
import cv2 as cv
img = cv.imread('image.jpg')
resized_img = cv.resize(img, (width, height))
cv.imwrite('resized_image.jpg', resized_img)
```

2024-07-23 21:29:29.238 | INFO     | langchain.callbacks.tracers.logging:callback:27 - [chain/end] [chain:AgentExecutor] [10.58s] Exiting Chain run with output:
[outputs]


To resize a JPEG image using OpenCV, you can use the `cv.resize()` function. Here's a detailed example in Python:

```python
import cv2 as cv

# Read the image
img = cv.imread('image.jpg')

# Define the new dimensions
width = 800  # Replace with your desired width
height = 600  # Replace with your desired height
new_dimensions = (width, height)

# Resize the image
resized_img = cv.resize(img, new_dimensions)

# Save the resized image
cv.imwrite('resized_image.jpg', resized_img)
```

In this example:
1. We read the image using `cv.imread()`.
2. We define the new dimensions for the image.
3. We resize the image using `cv.resize()`.
4. We save the resized image using `cv.imwrite()`.

Make sure to replace `'image.jpg'` with the path to your JPEG image and adjust the `width` and `height` to your desired dimensions.

For more detailed information, you can refer to the [OpenCV documentation](https://docs.opencv.org/4.x/da/d6e/tutorial_py_geometric_transformations.html).

> Finished chain.


In [11]:
# events

[{'event': 'on_chain_start',
  'data': {'input': {'input': "{'user name': 'bossjones', 'message': 'how do i resize a jpeg image using opencv?', 'surface_info': {'surface': 'discord', 'type': 'channel', 'source': 'gpt'}}"}},
  'name': 'AgentExecutor',
  'tags': [],
  'run_id': '',
  'metadata': {'session_id': '1111_test'},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': '',
  'metadata': {'session_id': '1111_test'},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableParallel<input,agent_scratchpad,chat_history>',
  'tags': ['seq:step:1'],
  'run_id': '',
  'metadata': {'session_id': '1111_test'},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableLambda',
  'tags': ['map:key:input'],
  'run_id': '',
  'metadata': {'session_id': '1111_test'},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableLambda',
  'tags': ['map:key:a

In [7]:
# async def test_with_explicit_config(agent) -> None:
#     """Test astream events with explicit callbacks being passed."""
#     infinite_cycle = cycle([AIMessage(content="hello world", id="ai3")])
#     model = GenericFakeChatModel(messages=infinite_cycle)

#     @tool
#     async def say_hello(query: str, callbacks: Callbacks) -> BaseMessage:
#         """Use this tool to look up which items are in the given place."""

#         @RunnableLambda
#         def passthrough_to_trigger_issue(x: str) -> str:
#             """Add passthrough to trigger issue."""
#             return x

#         chain = passthrough_to_trigger_issue | model.with_config(
#             {"tags": ["hello"], "callbacks": callbacks}
#         )

#         return await chain.ainvoke(query)

#     events = await _collect_events(
#         say_hello.astream_events("meow", version="v2")  # type: ignore
#     )

#     assert [
#         event["data"]["chunk"].content
#         for event in events
#         if event["event"] == "on_chat_model_stream"
#     ] == ["hello", " ", "world"]